<a href="https://colab.research.google.com/github/alexmarcel/simple-mnist-neural-net/blob/main/simple_mnist_neural_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [1]:
# imports

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from PIL import Image

Create Fully Connected Network

In [2]:
class NN(nn.Module):

    def __init__(self, input_size, num_classes):
      super(NN, self).__init__()

      self.fc1 = nn.Linear(input_size, 50)
      self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return x

Set Device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device : {device}")

Hyperparameters

In [4]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

Load Data

In [5]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Initialize Network

In [6]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

Loss and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Train Network

In [ ]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):

    # Move data to device (cuda/cpu)
    data = data.to(device=device)
    targets = targets.to(device=device)

    # Get to correct shape / flatten 28x28 to 784
    data = data.reshape(data.shape[0], -1)

    # Forward
    scores = model(data)
    loss = criterion(scores, targets)

    # Backward
    optimizer.zero_grad() # set gradient to zero for each batch
    loss.backward()

    # Gradient descent or adam step
    optimizer.step()

    print(f"Epoch : {epoch} - batch - {batch_idx} - loss : {loss}")


Check accuracy on training and test to see model performance

In [ ]:
def check_accuracy(loader, model):

  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0

  model.eval() # set model to eval mode

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f"{num_correct} / {num_samples} with accuracy of {float(num_correct)/float(num_samples)*100:.2f}")

  model.train() # set model to train mode, if need to train more

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Save Model

In [10]:
torch.save(model.state_dict(), 'model.pth')

Load Model

In [ ]:
model_test = NN(input_size, num_classes).to(device) # input_size and num_classes should follow trained model hyperparameters

model_test.load_state_dict(torch.load('model.pth'))

Test Loaded Model

In [ ]:
# Set to eval mode
model.eval()

# Define the transformation to apply to the input image
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure the image is grayscale if needed
    transforms.Resize((28, 28)),  # Resize the image to match the MNIST size
    transforms.ToTensor(),
])

# Load and preprocess the single image
image_path = 'number.jpg'
image = Image.open(image_path)
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# Perform inference
with torch.no_grad():
    output = model_test(input_batch.reshape(input_batch.shape[0], -1).to(device))

# Interpret the model output (assuming classification task)
_, predicted_class = torch.max(output, 1)

# Print the predicted class
print(f'Predicted class: {predicted_class.item()}')

In [ ]:
# Optionally, visualize the input image and the model's prediction
plt.imshow(image, cmap='gray')
plt.title(f'Predicted class: {predicted_class.item()}')
plt.show()